## Goal: aggregate local government programs at the city level

In [1]:
import pandas as pd

In [3]:
def no_dupes(df):
    
    #creating df of number of times a claim repeats

    repeats = df['ClaimID'].value_counts().to_frame().reset_index()

    #getting list of duplicate claims
    
    all_dupes = repeats[repeats['ClaimID']>1]
    dupes = all_dupes['index'].to_list()
   

    #getting one index value per duplicate claim

    goodi=[]
    for e in dupes:
        goodi.append(min(df[df['ClaimID'].isin([e])].index.to_list()))
    
    # full list of duplicate claim indicies

    alli=[]
    for e in dupes:
        alli = alli + df[df['ClaimID'].isin([e])].index.to_list()


    # take goodi from alli
    badi = [i for i in alli if i not in goodi]

    #drop duplicate indicies
    return(df.drop(index=badi).reset_index())

In [4]:
column_list = ['TotalFirstYearGrosskWh',
       'TotalFirstYearGrossTherm','TotalLifecycleGrosskWh',
       'TotalLifecycleGrossTherm','Budget']
def aggregate_me(df, by, lg, column_list):
    
    # making df usable

    df[by] = df[by].str.upper()
    
    #filtering for local government

    if lg: 

        lglist = ['Local Government (Counties)',
         'Local Government ()',
         'Local Government (Cities)',
         'Local Government ']

        df = df[df['Grouping'].isin(lglist)]


    # creating a total programs column

    prg = df.groupby([by, 'PrgID'], dropna = False).size().index.tolist() 
    #need to count the number of times a city/county shows up 

    e,f = [list(c) for c in zip(*prg)]

    ccp = pd.Series(e) # made a series so can use value_counts

    ccp = ccp.str.upper()

    tp = ccp.value_counts(dropna = False) #total programs per city
    
    # creating a total claims column
    
    df['Total Claims'] = [1]*len(df)
    
    #aggregate!
    relevant_columns = column_list + ['Total Claims']
    agg = df.groupby(by).sum()[relevant_columns]
    agg['Total Programs'] = tp
    
    # Resource/OBF budget as percentage of total budget
    obf_budget = df.groupby([by, 'OBF_Flag']).Budget.sum().unstack().fillna(0).loc[:, True]
    resource_budget = df.groupby([by, 'Resource_Flag']).Budget.sum().unstack().fillna(0).loc[:, 1]
    agg['% OBF Budget'] = (obf_budget/agg['Budget'])*100
    agg['% Resource Budget'] = (resource_budget/agg['Budget'])*100
    
    return(agg.reset_index())

In [6]:
lg = pd.read_csv('data/lg_with_budget_and_resource.csv')

C:\Users\audre\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (18,21,22,23,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
lg = lg.drop_duplicates()

In [11]:
lg.shape

(192092, 72)

In [12]:
local_gov = lg.copy()

In [13]:
local_gov = no_dupes(local_gov)

In [14]:
local_gov.shape

(192092, 73)

### Need to add county data

In [15]:
c = pd.read_csv('data/city_county_mapping.csv')

In [16]:
local_gov = local_gov.merge(c, left_on='SiteCity', right_on='city', how='left')

In [17]:
local_gov.columns

Index(['index', 'ClaimID', 'PrgID', 'Sector_x', 'SiteCity', 'SiteZipCode',
       'SiteID', 'NAICSCode', 'BldgHVAC', 'BldgLoc', 'BldgType', 'BldgVint',
       'E3ClimateZone', 'E3GasSavProfile', 'E3GasSector',
       'E3MeaElecEndUseShape', 'E3TargetSector', 'ImplementationID',
       'InstallationDate', 'NAICSBldgType', 'NumUnits', 'OBF_Flag',
       'PrgElement', 'RateScheduleElec', 'RateScheduleGas', 'REN_Flag',
       'Residential_Flag', 'SchoolIdentifier', 'TotalFirstYearGrosskW',
       'TotalFirstYearGrosskWh', 'TotalFirstYearGrossTherm',
       'TotalGrossIncentive', 'TotalGrossMeasureCost',
       'TotalGrossMeasureCost_ER', 'TotalLifecycleGrosskW',
       'TotalLifecycleGrosskWh', 'TotalLifecycleGrossTherm', 'Upstream_Flag_x',
       'WaterOnly_Flag', 'PrgYear', 'Claim ID', 'Program ID',
       'First Year Gross kWh', 'First Year Gross Therm', 'Budget', 'PA',
       'ProgramName', 'Grouping', 'County', 'Selected', 'PrimarySector',
       'Sector_y', 'BP Sector', 'ProgramImple

### Now for the aggregation!

In [18]:
county_ag = aggregate_me(local_gov, 'county', False, column_list)

In [19]:
county_ag.head()

,county,TotalFirstYearGrosskWh,TotalFirstYearGrossTherm,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,Budget,Total Claims,Total Programs,% OBF Budget,% Resource Budget
0,ALAMEDA,1.313266e+07,-55166.943648,1.375638e+08,-572536.128952,8.383464e+06,24202,7,21.651553,100.0
1,AMADOR,6.303598e+05,-4445.044561,6.969587e+06,-47535.941272,5.484184e+05,129,2,0.000000,100.0
2,BUTTE,1.786076e+06,-2434.795903,2.032414e+07,-26140.758805,1.665998e+06,5011,5,16.661132,100.0
3,CALAVERAS,5.730568e+04,-407.880660,6.178263e+05,-4790.258736,4.330963e+04,18,2,3.253734,100.0
4,COLUSA,1.957298e+05,-478.134807,1.949328e+06,-5486.333745,1.150678e+05,11,2,18.890025,100.0


In [23]:
local_gov.Grouping.value_counts(dropna=False)

Local Government (Counties)    168740
Local Government ()             21392
Local Government (Cities)        1947
Local Government                   13
Name: Grouping, dtype: int64

# whooo it worked big party

now going to make the complete dataset, add demographic info

In [28]:
dems = pd.read_csv('data/county_demographics.csv')

In [29]:
county_ag.head()

,county,TotalFirstYearGrosskWh,TotalFirstYearGrossTherm,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,Budget,Total Claims,Total Programs,% OBF Budget,% Resource Budget
0,ALAMEDA,1.313266e+07,-55166.943648,1.375638e+08,-572536.128952,8.383464e+06,24202,7,21.651553,100.0
1,AMADOR,6.303598e+05,-4445.044561,6.969587e+06,-47535.941272,5.484184e+05,129,2,0.000000,100.0
2,BUTTE,1.786076e+06,-2434.795903,2.032414e+07,-26140.758805,1.665998e+06,5011,5,16.661132,100.0
3,CALAVERAS,5.730568e+04,-407.880660,6.178263e+05,-4790.258736,4.330963e+04,18,2,3.253734,100.0
4,COLUSA,1.957298e+05,-478.134807,1.949328e+06,-5486.333745,1.150678e+05,11,2,18.890025,100.0


In [30]:
dems.head()

,County,Population,Mean Income,CES Score Median,IRR,TotalRevenue,dac_proportion,ces_score_median,ces_percentile_median,median_household_income_usd
0,ALAMEDA,1656754,76626,21.532906,0.270123,9617053146,0.105556,21.532906,42.098625,99406
1,ALPINE,1039,78945,11.343357,0.656045,60490797,0.000000,11.343357,15.853197,63750
2,AMADOR,38429,51453,23.257549,0.502563,243032260,0.000000,23.257549,45.756085,62772
3,BUTTE,225817,62405,20.033261,0.434555,1290504922,0.039216,20.033261,38.365494,52537
4,CALAVERAS,45514,70651,15.180852,0.509595,677652486,0.000000,15.180852,25.551772,63158


In [35]:
counties_1 = dems.merge(county_ag, left_on='County', right_on='county', how='left')

In [36]:
counties_1.shape

(58, 20)

In [37]:
dems.shape

(58, 10)

In [38]:
county_ag.shape

(51, 10)

In [40]:
counties_1[counties_1.county.isna()]

,County,Population,Mean Income,CES Score Median,IRR,TotalRevenue,dac_proportion,ces_score_median,ces_percentile_median,median_household_income_usd,county,TotalFirstYearGrosskWh,TotalFirstYearGrossTherm,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,Budget,Total Claims,Total Programs,% OBF Budget,% Resource Budget
1,ALPINE,1039,78945,11.343357,0.656045,60490797,0.000000,11.343357,15.853197,63750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,DEL NORTE,27495,62144,16.844851,0.530593,225894688,0.000000,16.844851,29.978560,45283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,IMPERIAL,180701,65273,39.583822,0.483676,1305614242,0.483871,39.583822,75.387817,47622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,LASSEN,30818,46389,15.820526,0.562445,206480533,0.000000,15.820526,27.355278,56352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,MODOC,8907,57068,16.525398,0.609696,96893007,0.000000,16.525398,29.253374,45507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,SIERRA,3040,61491,19.965483,0.614994,65142974,0.000000,19.965483,38.201539,52148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,TRINITY,12700,46949,8.569741,0.585870,173811393,0.000000,8.569741,9.837306,40846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
counties_1.fillna(0, inplace=True)

In [49]:
counties_1.drop(columns='county', inplace=True)

In [53]:
counties_1.drop(columns='CES Score Median', inplace=True)

In [54]:
counties_1.to_csv('data/counties_1.csv',index=False)

In [56]:
len(counties_1[counties_1.Budget==0])

7